# load data

In [1]:
from glob import glob
import os
images=[]
labels=[]
for folder in os.listdir('../dataset/wrist_xray'):
    for pic in glob('../dataset/wrist_xray/{}/*.jpg'.format(folder)):
        images.append(pic)
        labels.append(folder)
        

In [2]:
import pandas as pd
data=pd.DataFrame(zip(images,labels),columns=['img','label'])
data.label=data.label.map({'Fracture':0,'Normal':1})
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,img,label
0,../dataset/wrist_xray/Fracture\65 Male (L View...,0
1,../dataset/wrist_xray/Fracture\40 F (A View).jpg,0
2,../dataset/wrist_xray/Normal\26.jpg,1
3,../dataset/wrist_xray/Fracture\57 F (A View).jpg,0
4,../dataset/wrist_xray/Fracture\63 Male (A View...,0


# top 5 lightest models in torchvision
1	squeezenet1_1	1235496  
2	shufflenet_v2_x0_5	1366792  
3	mnasnet0_5	2218512  
4	mobilenet_v2	3504872  
5	densenet121	7978856  `

In [3]:
import sys
sys.path.append("../src") 

In [4]:
from models.squeezenet import squeezenet1_1
from models.mymodels import mymodel4
from models.shufflenet import shufflenet_v2_x0_5
from models.mnastnet import mnasnet0_5
from models.densenet import densenet121
from models.mobilenet import mobilenet
from cross_vals import kfoldcv

In [5]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)/1e6

In [6]:
print('MyNet parameters :' ,count_parameters(mymodel4()))
print('SqueezeNet parameters :', count_parameters(squeezenet1_1()))
print('ShuffleNet parameters :', count_parameters(shufflenet_v2_x0_5()))
print('MnastNet parameters :', count_parameters(mnasnet0_5()))
print('MobileNet parameters :', count_parameters(mobilenet()))
print('DenseNet parameters :', count_parameters(densenet121()))


MyNet parameters : 0.473328
SqueezeNet parameters : 0.787008
ShuffleNet parameters : 0.47256
MnastNet parameters : 1.101192
MobileNet parameters : 2.387264
DenseNet parameters : 7.472384


In [7]:
epoch=32
batchsize=8
lr=0.001

dim='3D' or '2D' for 3D images and 2D iamges
skip_tuning=False, no grid search applied for hyper-parameter tuning, 
aug=1-> pytorch augmentation
aug=0-> no augmentation
aug=2-> albumentation augmentation
aug=3-> rand augmentation

# MyNet

In [8]:
%%time
train_cv,clf_report=kfoldcv(model=mymodel4(),epochs=epoch,batchsize=batchsize,data=data,\
                            lr=0.001,skip_tuning=False,aug=1,dim='2D',pca=False)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.900   0.808     0.836
Class 1       0.772   0.882     0.806
macro         0.836   0.846     0.820
weighted      0.864   0.822     0.826
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.896   0.876     0.876
Class 1       0.830   0.868     0.838
macro         0.862   0.870     0.858
weighted      0.880   0.858     0.862
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.896   0.866     0.872
Class 1       0.820   0.868     0.832
macro         0.858   0.866     0.852
weighted      0.876   0.854     0.858
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.904   0.876     0.880
Class 1       0.832   0.884     0.846
macro         0.868   0.878     0.864
weighted      0.888   0.864     0.866
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.74,0.79,0.79,0.79
1,0.82,0.87,0.85,0.87
2,0.82,0.82,0.82,0.82
3,0.84,0.89,0.89,0.92
4,0.89,0.92,0.92,0.92
mean,0.822,0.858,0.854,0.864


In [9]:
%%time
train_cv,clf_report=kfoldcv(model=mymodel4(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.636   0.712     0.658
Class 1       0.536   0.436     0.456
macro         0.584   0.578     0.554
weighted      0.614   0.582     0.574
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.820   0.764     0.784
Class 1       0.706   0.778     0.732
macro         0.760   0.772     0.758
weighted      0.786   0.762     0.766
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.844   0.798     0.812
Class 1       0.742   0.808     0.762
macro         0.796   0.802     0.790
weighted      0.816   0.794     0.794
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.772   0.654     0.690
Class 1       0.614   0.742     0.652
macro         0.696   0.696     0.672
weighted      0.724   0.674     0.676
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.49,0.67,0.77,0.67
1,0.51,0.74,0.77,0.56
2,0.54,0.74,0.72,0.69
3,0.61,0.84,0.84,0.74
4,0.76,0.82,0.87,0.71
mean,0.582,0.762,0.794,0.674


# SqueezeNet

In [10]:
%%time
train_cv,clf_report=kfoldcv(model=squeezenet1_1(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.652   0.602     0.616
Class 1       0.500   0.564     0.518
macro         0.576   0.584     0.566
weighted      0.600   0.576     0.574
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.656   0.746     0.684
Class 1       0.542   0.440     0.456
macro         0.598   0.596     0.568
weighted      0.624   0.600     0.582
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.700   0.782     0.724
Class 1       0.602   0.520     0.530
macro         0.650   0.652     0.626
weighted      0.672   0.652     0.636
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.658   0.726     0.678
Class 1       0.530   0.468     0.466
macro         0.594   0.598     0.570
weighted      0.620   0.600     0.584
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.90,0.87,0.87,0.87
1,0.49,0.46,0.51,0.56
2,0.44,0.51,0.49,0.51
3,0.50,0.61,0.68,0.53
4,0.55,0.55,0.71,0.53
mean,0.576,0.6,0.652,0.6


In [11]:
%%time
train_cv,clf_report=kfoldcv(model=squeezenet1_1(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.644   0.598     0.614
Class 1       0.488   0.548     0.508
macro         0.566   0.574     0.562
weighted      0.592   0.584     0.582
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.614   0.618     0.602
Class 1       0.488   0.480     0.462
macro         0.550   0.550     0.530
weighted      0.576   0.542     0.542
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.606   0.544     0.570
Class 1       0.456   0.528     0.486
macro         0.532   0.536     0.528
weighted      0.556   0.534     0.538
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.620   0.546     0.570
Class 1       0.452   0.528     0.474
macro         0.534   0.536     0.522
weighted      0.562   0.528     0.530
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.79,0.77,0.77,0.72
1,0.54,0.38,0.46,0.44
2,0.54,0.56,0.44,0.51
3,0.58,0.47,0.50,0.50
4,0.47,0.53,0.50,0.47
mean,0.584,0.542,0.534,0.528


# ShuffleNet

In [12]:
%%time
train_cv,clf_report=kfoldcv(model=shufflenet_v2_x0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.654   0.656     0.640
Class 1       0.552   0.522     0.502
macro         0.604   0.590     0.570
weighted      0.628   0.590     0.586
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.662   0.760     0.700
Class 1       0.604   0.482     0.516
macro         0.632   0.622     0.608
weighted      0.650   0.634     0.622
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.642   0.778     0.698
Class 1       0.596   0.428     0.482
macro         0.620   0.602     0.590
weighted      0.636   0.624     0.608
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.680   0.790     0.722
Class 1       0.636   0.504     0.542
macro         0.660   0.648     0.630
weighted      0.676   0.654     0.644
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.49,0.54,0.54,0.59
1,0.64,0.74,0.72,0.77
2,0.56,0.54,0.54,0.54
3,0.63,0.74,0.71,0.76
4,0.63,0.61,0.61,0.61
mean,0.59,0.634,0.624,0.654


In [13]:
%%time
train_cv,clf_report=kfoldcv(model=shufflenet_v2_x0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.478   0.476     0.466
Class 1       0.330   0.318     0.314
macro         0.402   0.400     0.390
weighted      0.426   0.406     0.408
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.612   0.744     0.664
Class 1       0.476   0.344     0.388
macro         0.544   0.546     0.526
weighted      0.568   0.572     0.550
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.600   0.668     0.622
Class 1       0.436   0.386     0.392
macro         0.518   0.528     0.504
weighted      0.542   0.536     0.518
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.646   0.698     0.668
Class 1       0.536   0.490     0.506
macro         0.592   0.592     0.588
weighted      0.612   0.606     0.604
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.28,0.54,0.49,0.62
1,0.44,0.49,0.54,0.56
2,0.46,0.46,0.41,0.46
3,0.32,0.66,0.61,0.76
4,0.53,0.71,0.63,0.63
mean,0.406,0.572,0.536,0.606


# Mnasnet

In [14]:
%%time
train_cv,clf_report=kfoldcv(model=mnasnet0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.812   0.724     0.762
Class 1       0.682   0.790     0.726
macro         0.744   0.758     0.744
weighted      0.768   0.746     0.750
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.802   0.754     0.776
Class 1       0.646   0.720     0.680
macro         0.726   0.736     0.726
weighted      0.746   0.746     0.744
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.818   0.796     0.800
Class 1       0.676   0.720     0.686
macro         0.748   0.758     0.744
weighted      0.768   0.774     0.764
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.814   0.776     0.790
Class 1       0.688   0.746     0.706
macro         0.750   0.764     0.752
weighted      0.774   0.764     0.762
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.51,0.46,0.59,0.59
1,0.90,0.95,0.95,0.90
2,0.72,0.77,0.77,0.77
3,0.84,0.84,0.82,0.82
4,0.76,0.71,0.74,0.74
mean,0.746,0.746,0.774,0.764


In [15]:
%%time
train_cv,clf_report=kfoldcv(model=mnasnet0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.570   0.634     0.596
Class 1       0.428   0.368     0.390
macro         0.500   0.500     0.492
weighted      0.520   0.516     0.510
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.598   0.618     0.604
Class 1       0.472   0.450     0.456
macro         0.534   0.532     0.530
weighted      0.554   0.548     0.550
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.580   0.694     0.626
Class 1       0.452   0.336     0.374
macro         0.516   0.518     0.502
weighted      0.538   0.544     0.526
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.626   0.572     0.590
Class 1       0.482   0.538     0.500
macro         0.554   0.554     0.548
weighted      0.580   0.560     0.562
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.51,0.51,0.64,0.54
1,0.64,0.64,0.67,0.62
2,0.41,0.56,0.46,0.67
3,0.63,0.61,0.58,0.50
4,0.39,0.42,0.37,0.47
mean,0.516,0.548,0.544,0.56


# MobileNet

In [16]:
%%time
train_cv,clf_report=kfoldcv(model=mobilenet(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.832   0.746     0.772
Class 1       0.714   0.786     0.736
macro         0.772   0.768     0.752
weighted      0.790   0.756     0.758
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.882   0.706     0.764
Class 1       0.714   0.874     0.770
macro         0.796   0.788     0.764
weighted      0.818   0.772     0.770
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.834   0.776     0.790
Class 1       0.746   0.812     0.762
macro         0.788   0.792     0.774
weighted      0.810   0.778     0.778
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.890   0.790     0.822
Class 1       0.770   0.866     0.806
macro         0.830   0.828     0.812
weighted      0.844   0.818     0.816
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.51,0.56,0.59,0.77
1,0.79,0.85,0.85,0.87
2,0.77,0.79,0.77,0.77
3,0.92,0.92,0.92,0.92
4,0.79,0.74,0.76,0.76
mean,0.756,0.772,0.778,0.818


In [17]:
%%time
train_cv,clf_report=kfoldcv(model=mobilenet(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.580   0.630     0.602
Class 1       0.426   0.366     0.388
macro         0.502   0.500     0.494
weighted      0.530   0.528     0.524
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.610   0.654     0.630
Class 1       0.480   0.434     0.452
macro         0.542   0.542     0.540
weighted      0.568   0.566     0.564
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.634   0.726     0.674
Class 1       0.510   0.412     0.454
macro         0.570   0.570     0.564
weighted      0.598   0.602     0.594
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.580   0.660     0.614
Class 1       0.464   0.360     0.398
macro         0.522   0.508     0.508
weighted      0.546   0.546     0.538
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.54,0.59,0.62,0.49
1,0.51,0.44,0.44,0.46
2,0.56,0.51,0.69,0.46
3,0.45,0.63,0.58,0.58
4,0.58,0.66,0.68,0.74
mean,0.528,0.566,0.602,0.546


# DenseNet121

In [18]:
%%time
train_cv,clf_report=kfoldcv(model=densenet121(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.770   0.738     0.744
Class 1       0.652   0.696     0.662
macro         0.712   0.718     0.706
weighted      0.736   0.712     0.714
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.806   0.744     0.762
Class 1       0.688   0.762     0.710
macro         0.746   0.750     0.736
weighted      0.772   0.740     0.742
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.814   0.784     0.788
Class 1       0.722   0.756     0.726
macro         0.768   0.770     0.758
weighted      0.790   0.766     0.766
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.822   0.724     0.762
Class 1       0.680   0.796     0.722
macro         0.750   0.758     0.744
weighted      0.776   0.744     0.748
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.82,0.87,0.87,0.87
1,0.62,0.67,0.74,0.69
2,0.64,0.69,0.72,0.69
3,0.66,0.71,0.74,0.68
4,0.82,0.76,0.76,0.79
mean,0.712,0.74,0.766,0.744


In [19]:
%%time
train_cv,clf_report=kfoldcv(model=densenet121(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.742   0.782     0.756
Class 1       0.674   0.630     0.644
macro         0.710   0.708     0.700
weighted      0.724   0.716     0.714
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.772   0.854     0.802
Class 1       0.764   0.668     0.700
macro         0.766   0.758     0.750
weighted      0.784   0.762     0.758
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.784   0.840     0.804
Class 1       0.752   0.696     0.710
macro         0.770   0.766     0.760
weighted      0.784   0.772     0.768
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.700   0.690     0.688
Class 1       0.616   0.618     0.606
macro         0.656   0.654     0.646
weighted      0.674   0.664     0.662
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.74,0.79,0.87,0.85
1,0.69,0.77,0.72,0.67
2,0.62,0.72,0.72,0.49
3,0.74,0.82,0.79,0.55
4,0.79,0.71,0.76,0.76
mean,0.716,0.762,0.772,0.664
